# Making a Mockery of Testing in Python

Testing objects is usually easy, but testing systems is hard, especially when each system has dependencies on external resources that you can't control. One of the most useful tools for testing multi-layered systems are **Mock objects**.

The basic idea of a mock object is that it acts *like* the thing you need but that isn't what you're testing, without actually being that thing. It *mocks* something you need to perform your test.

The quintessential example is mocking a database... you have a database connection and you want to run queries against it but you want to be sure you always get the response you want without having to maintain a full database that everyone keeps in sync over time.

## Getting started

The `mock` library is part of the `unittest` package in core Python (and a top-level package of Python 2):

In [1]:
from unittest import mock  # Python 3
# import mock  # Python 2

## Your first mock object

The easiest mock object to start with is the `MagicMock`. Simply create one and interact with it:

In [2]:
my_doohicky = mock.MagicMock()

You can now call methods against your object. Any method you call will exist, and any property you access will also exist:

In [3]:
my_doohicky.some_function()

<MagicMock name='mock.some_function()' id='4481786264'>

In [4]:
my_doohicky.some_property

<MagicMock name='mock.some_property' id='4481821048'>

You can tell the mock to return a certain value for a method, and then call it to retrieve that value:

In [5]:
my_doohicky.some_function = mock.MagicMock(return_value=42)
my_doohicky.some_function()

42

You can also confirm the method was called a certain way:

In [6]:
my_doohicky.some_function.assert_called_once()
my_doohicky.some_function.assert_called_with()
# ^^ those pass

In [7]:
# this one fails:
# my_doohicky.some_function.assert_called_once_with('abc', y=42)

In [8]:
# let's try to call it again, this time testing the arguments are correct:
my_doohicky.some_function.reset_mock()
my_doohicky.some_function('abc', y=42)
my_doohicky.some_function.assert_called_once_with('abc', y=42)

In [9]:
# let's change the order of things and see what happens
my_doohicky.some_function.reset_mock()
my_doohicky.some_function('abc', x=1, y=42, z=6)
my_doohicky.some_function.assert_called_once_with('abc', z=6, x=1, y=42)

## Example: testing function a() calls function b()

Here's how you can confirm your inner function works as expected:

In [10]:
class MyClass(object):
    
    def multiply(self, x, y):
        return x * y

    def square(self, x):
        return self.multiply(x, x)

class_to_test = MyClass()
class_to_test.multiply = mock.MagicMock()
class_to_test.square(5)
class_to_test.multiply.assert_called_once_with(5, 5)

## Example: testing the consumer of a function that hasn't been written

Here's how you can test the function you *have* written before you're done writing a function you haven't figured out yet:

In [11]:
import json

class MyIncompleteClass(object):
    
    def load_from_web(self, url):
        raise NotImplementedError()
    
    def convert_to_json(self, url):
        file = self.load_from_web(url)
        return json.loads(file)

to_test = MyIncompleteClass()
to_test.load_from_web = mock.MagicMock(return_value='{"x": 1, "y": 2}')
value = to_test.convert_to_json('http://google.com/')
to_test.load_from_web.assert_called_with('http://google.com/')
value

{'x': 1, 'y': 2}

## Resources

* [Python's docs on `mock`](https://docs.python.org/3/library/unittest.mock.html)
* [Python Mocking 101: Fake It Before You Make It](https://blog.fugue.co/2016-02-11-python-mocking-101.html)